### Validación del Modelo con Aumento de Datos

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import LeaveOneOut


In [3]:
# Cargar el archivo CSV con los datos procesados en un DataFrame de pandas
data = pd.read_csv('../data/new_data.csv')

In [4]:
# Separar las características de la variable objetivo en el DataFrame
X,y = data.drop('Estado al egreso', axis=1), data['Estado al egreso']

In [5]:
import pickle as pkl

# Cargar el modelo Random Forest previamente guardado desde el archivo .pkl
with open('../models/new_rf.pkl', 'rb') as file:
    best_rf = pkl.load(file)

El análisis del conjunto de datos ampliado aplicó SMOTE (Synthetic Minority Oversampling Technique) para balancear las clases, manteniendo la consistencia metodológica con el proceso original de selección de modelos y utilizando el modelo Random Forest preentrenado optimizado con las 5 características clave.

In [6]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score, precision_score, recall_score

loo = LeaveOneOut()
y_true = []
y_pred = []

# Validación Leave-One-Out con aplicación dinámica de SMOTE en cada iteración de entrenamiento
for train_idx, test_idx in loo.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Contar cantidad de muestras de la clase minoritaria en el conjunto de entrenamiento
    n_minority = min(Counter(y_train).values())

    # Ajustar k_neighbors para SMOTE según el número de muestras minoritarias (mínimo 5 o menor si hay pocas muestras)
    k = min(5, n_minority - 1) 
    
    # Aplicar SMOTE solo si hay suficientes muestras minoritarias para generar sintéticos
    if n_minority > 1 and k > 0:
        smote = SMOTE(k_neighbors=k, random_state=42)
        X_res, y_res = smote.fit_resample(X_train, y_train)
    else:
        # Si no hay suficientes muestras, usar los datos originales sin sobremuestreo
        X_res, y_res = X_train, y_train
    
    # Entrenar el modelo Random Forest con los datos balanceados
    model = best_rf
    model.fit(X_res, y_res)

    # Predecir la muestra de prueba
    y_pred_i = model.predict(X_test)
    
    # Guardar etiquetas reales y predichas para evaluación posterior
    y_true.append(y_test.values[0])
    y_pred.append(y_pred_i[0])

# Calcular métricas
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred)

print("\n" + "="*50)
print(f"Resultados con SMOTE (5 features):")
print(f"F1: {f1:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}")
print("="*50)


Resultados con SMOTE (5 features):
F1: 0.963, Precision: 0.929, Recall: 1.000


Los resultados demostraron métricas de desempeño comparables entre el modelo original y el modelo aumentado con SMOTE (F1 ~0.96), lo que indica que:

- El modelo ya era robusto: el Random Forest original (5 características) generalizó bien sin necesidad de datos sintéticos.

- Beneficio limitado del sobremuestreo: dado que el rendimiento no mejoró, es probable que los datos de entrenamiento iniciales capturaran suficientemente los patrones clínicos subyacentes.

- Fronteras de decisión estables: la lógica predictiva del modelo se mantuvo consistente incluso con la ampliación de los conjuntos de entrenamiento, lo que sugiere que no depende excesivamente de puntos de datos específicos.
